In [28]:
import os
import re
from tika import parser

In [29]:
##### Scraping Don Quijote

pdf_dir = '/Users/eun-youngchristinapark/Documents/UBC-CAPSTONE/pdf_scraping/'
donquijote = 'Recorridos_Don-Quijote-Student-text-2017.pdf'
file_name = pdf_dir + donquijote

In [41]:
#### Get the raw text
raw = parser.from_file(file_name)
raw_list = raw['content'].strip().split('\n\nRecorridos 3—Siglo XVI     Cervantes ')  ## contains pages of the book
text_list = []
regex = r'[0-9]+\. '
regex2 = r'[0-9]+\-'
regex3 = r'Capítulo [0-9]+  \n\n'
regex4 = r'[0-9]+\.'
regex5 = r'\n+\s+[0-9]+\s+\n+'

# extract text only - remove questions / discussions / vocabularies etc. 
# text begins at page 12 and ends at page 85

for i in range(11,84):
    if 'BANCO DE PALABRAS ÚTILES' not in raw_list[i]:  # skip End-of-Chapter Questions 
        
        ### Get the text boundary
        
        if re.search(regex3, raw_list[i]) == None:
            begin_index = raw_list[i].find('\n\n')
            end_index = raw_list[i][:raw_list[i].find('=')+1].rfind('\n\n')
            end_index_prev = 0 
            if raw_list[i][begin_index:begin_index+30].startswith('\n\n¿COMPRENDISTE LO QUE LEÍSTE?'):            # skip the end of chapter question section 
                for match in re.finditer(regex4, raw_list[i]):
                    begin_index = match.span()[0]
                begin_index = raw_list[i].find('\n\n', begin_index + 1)
                end_index = raw_list[i][begin_index+1:raw_list[i].find('=')+1].rfind('\n\n') + begin_index+1
                
            elif begin_index == end_index:                                                                        # the page starts with vocabulary list: we have to skip this section
                while end_index != -1:
                    end_index_prev += end_index + 1
                    end_index = raw_list[i][end_index_prev+1:].find('=')
                begin_index = raw_list[i].find('\n\n', end_index_prev + 1)
                end_index = 1
                search_begin_index = begin_index 
                while end_index > 0:
                    regex_match = re.search(regex5, raw_list[i][search_begin_index + 1:])
                    if regex_match:
                        end_index = regex_match.span()[0]
                        search_begin_index += end_index 
                    else:
                        end_index = -1 
                end_index = search_begin_index
        else:
            begin_index = re.search(regex3, raw_list[i]).span()[0]
            end_index = raw_list[i][begin_index+1:raw_list[i].find('=')+1].rfind('\n\n') + begin_index+1
        
        
        # For each sentence, determine whether the sentence is in the text 
        
        ts = raw_list[i][begin_index:end_index]   # main text 
        ts_list = ts.split('\n')
        ts_list_filtered = []
        qu_list = [0] * len(ts_list)
        for idx, sent in enumerate(ts_list):
            qu_list[idx] = 0
            if sent.strip() == '' and idx > 0:                                                                        # remove empty strings 
                qu_list[idx] = qu_list[idx-1]
            elif sent.strip().startswith('¿') or re.match(regex, sent.strip()) or re.match(regex2, sent.strip()):     # textbook questions or comments
                if sent.strip().endswith('.') or sent.strip().endswith('?') or re.match(regex2, sent.strip()):
                    qu_list[idx] = 1                                                                                  # question started and ended
                else:
                    qu_list[idx] = 2                                                                                  # question started (not ended)
            elif idx > 0 and (qu_list[idx-1] == 2 or qu_list[idx-1] == 3):
                if sent.strip().endswith('.') or sent.strip().endswith('?'):
                    qu_list[idx] = 4                                                                                  # question ended
                else:
                    qu_list[idx] = 3                                                                                  # question continues 
    
            elif sent.strip().startswith('“'):
                if sent.strip().endswith('”'):
                    qu_list[idx] = 5                                                                                  # quotation started and ended
                else:
                    qu_list[idx] = 6                                                                                  # quotation started (not ended)
            elif idx > 0 and qu_list[idx-1] == 6 and not(sent.strip().endswith('”')):
                qu_list[idx] = 7                                                                                      # quotatoin continues
            elif idx > 0 and (qu_list[idx-1] == 6 or qu_list[idx-1] == 7) and sent.strip().endswith('”'):
                qu_list[idx] = 8                                                                                      # quotation ended 
            if idx > 0 and qu_list[idx-1] == 8:
                qu_list[idx] = 9
        
            if qu_list[idx] == 0:
                ts_list_filtered.append(sent)
        ts_filtered = '\n'.join(ts_list_filtered)
        text_list.append(ts_filtered)

In [68]:
i = 25
print(text_list[i].strip())

Un mozo de mulas de los que allí venían, que no debía de ser 
muy bien intencionado, oyendo decir al pobre caído tantas    
arrogancias, no lo pudo sufrir sin darle la respuesta en las 
costillas. Y, llegándose a él, tomó la lanza y, después de haberla 
hecho pedazos, con uno dellos comenzó a dar a nuestro don 
Quijote tantos palos que, a despecho y pesar de sus armas, le molió 
como cibera. Débanle voces sus amos que no le diese tanto y que 
le dejase, pero estaba ya el mozo picado y no quiso dejar el juego 
hasta envidar todo el resto de su cólera; y, acudiendo por los 
demás trozos de la lanza, los acabó de deshacer sobre el miserable 
caído, que, con toda aquella tempestad de palos que sobre él 
vía, no cerraba la boca, amenazando al cielo y a la tierra, y a los 
malandrines, que tal le parecían. 
 
  Cansóse el mozo, y los mercaderes siguieron su camino, 
llevando qué contar en todo él del pobre apaleado. El cual, después 
que se vio solo, tornó a probar si podía levantarse; pero 